# S&P500 Stock Price Indicator
Group Members: Pranav Chavali, Tejas Iyer, Pranav Peddamalla, Adithya Pradosh

# Introduction
The main purpose of this project is to journey through the entirety of the data science life cycle. To accomplish this we have chosen to use a data set which keeps track of S&P500 stock prices from 2014 to 2017 with an attempt to create a predictor/indicator for a particular stock. Stock trading is a prevalent occupation around the world and traders use a plethora of indicators such as RSI (relative strength index), ADI (Average directional index), and many others

This is important because it has direct effects on our economy. If an indicator with high accuracy can be made this will benefit both businesses and the trader as they both will be making profits. We have also seen the negative side to this where people lose their money and we want to prevent this by creating an accurate indicator. This can be accomplished by using a large dataset which displays the historical changes for a given stock. By analyzing past trends in the market, we can begin to predict future market growth and create indicators that better fit to our data

Although, we note that it is important to consider that stock predictors are seldom as accurate as they are intended to be. After all, if it was easy to predict the price of stocks, everyone would be a millionaire. 

There are a number of different factors that play into affecting the price of a stock. A number of these factors are intangible, and unable to be measured or manipulated as quantitative data. One example of this is news regarding a stock: when new information regarding a company is released to the public, it can have a significant impact on its stock price. After all, the price of a stock directly reflects the trust and confidence that investors have in a given product at a given time. So if news comes out which influences an investors thoughts on whether to hold or sell, this can be directly reflected in the stock price as investor’s confidence in a stock beings to either waver or intensify. 

With this being said, our project intends to compare a number of well performing stocks from the S&P500, an index containing the top 500 leading publicly-traded companies in the USA to each other, as well as the S&P500 itself. Afterwords, we train a model using [INSERT MODEL HERE] to predict future stock price trends for a select few companies. 

For the companies, we chose to narrow down on 4 companies which are Apple, Starbucks, Amazon and Microsoft. These are very well known companies which majority of people know about. Apple and Microsoft are leading tech companies, Starbucks is one of the worlds leading coffee shops and Amazon is the leading e-commerce store in the world.

Throughout this project, we will go through the following stages (each item in this list will link to its respective code segments)
1. Data Collection
2. Data Processing
3. Exploratory Analysis & Data Visualization
4. Model: Analysis, Hypothesis, & ML
5. Interpretation: Insight & Policy Decision

In [ ]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.dates as mdates

This code performs data collection and preprocessing for multiple stocks, including S&P 500, Apple, Starbucks, Amazon, and Microsoft. It reads CSV files containing historical stock price data for each company and removes any rows with missing values using dropna. It also scrapes the S&P 500 historical prices table from a website and stores the data in a pandas DataFrame called sp_data. Overall, this code aims to gather and clean stock price data for further analysis or modeling.

#Data Collection:

Data Collection for Explanatory Analysis and Data Visualization: [EXPLAIN CODE HERE]

In [ ]:
# s&p data
sp = pd.read_csv('stock prices.csv')
sp = sp.dropna()

# apple data
apple = pd.read_csv('apple five year data.csv')
apple = apple.dropna()

# starbucks data
starbucks = pd.read_csv('starbucks five year data.csv')
starbucks = starbucks.dropna()

# amazon data
amazon = pd.read_csv('amazon five year data.csv')
amazon = amazon.dropna()

# microsoft data
microsoft = pd.read_csv('microsoft five year data.csv')
microsoft = microsoft.dropna()

# web scraping s&p changes over years
url = 'https://www.multpl.com/s-p-500-historical-prices/table/by-month'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    'From': 'stopdenyingme@gmail.com'
}
r = requests.get(url, headers=headers)

page = BeautifulSoup(r.content, 'lxml')
sp_data = pd.read_html(str(page.find('table')))[0]
sp_data.columns = ['Date', 'Price']
sp_data.index = sp_data.index + 1

Data Collection For Machine Learning: [Explain Code Here]

Apple Data:

In [ ]:
# Get the Dataset
dataset = pd.read_csv('HistoricalData_1683500505754.csv')

Amazon Data:

In [ ]:
# Get the Dataset
dataset2 = pd.read_csv('HistoricalData_1683662795655.csv')

# Data Preprocessing:

The code here performs are more extensive analysis on the data we collected in the previous cell. We take the data from each company and construct plots/tables about specific aspects regarding each company. For the plots, we plot the average close/last price over the last 5 years to see the companies overall improvement. In the tables at the end, we show each companies close/last price currently and their market cap which provides us more insights regarding the company. 

Data Pre Processing For Explanatory Data Analysis and Data Visualization:

In [ ]:
# all the pre processing done for Apple data set
apple_dates = list(apple['Date'])
apple_years = []

# adding all the years stripping out the MM/DD/ part of the date
for date in apple_dates:
  apple_years.append(date[6::])

# adding the new column to data set
apple['Year'] = apple_years
apple = apple[['Year', 'Date', 'Close/Last', 'Volume', 'Open', 'High', 'Low']]

# getting all the average close/last prices and storing in an array
apple_avgs = []
for year in sorted(set(apple_years)):
  total = 0
  # temp data set which corresponds to each year in the 5 year average
  temp = apple.loc[apple['Year'] == str(year)]
  close_last = list(temp['Close/Last'])
  for val in close_last:
    total += float(val[1::])

  apple_avgs.append(total / len(close_last))

In [ ]:
# all the pre processing done for Starbucks data set
starbucks_dates = list(starbucks['Date'])
starbucks_years = []

# adding all the years stripping out the MM/DD/ part of the date
for date in starbucks_dates:
  starbucks_years.append(date[6::])

# adding the new column to data set
starbucks['Year'] = starbucks_years
starbucks = starbucks[['Year', 'Date', 'Close/Last', 'Volume', 'Open', 'High', 'Low']]

# getting all the average close/last prices and storing in an array
starbucks_avgs = []
for year in sorted(set(starbucks_years)):
  total = 0
  # temp data set which corresponds to each year in the 5 year average
  temp = starbucks.loc[starbucks['Year'] == str(year)]
  close_last = list(temp['Close/Last'])
  for val in close_last:
    total += float(val[1::])

  starbucks_avgs.append(total / len(close_last))

In [ ]:
# all the pre processing done for Amazon data set
amazon_dates = list(amazon['Date'])
amazon_years = []

# adding all the years stripping out the MM/DD/ part of the date
for date in amazon_dates:
  amazon_years.append(date[6::])

# adding the new column to data set
amazon['Year'] = amazon_years
amazon = amazon[['Year', 'Date', 'Close/Last', 'Volume', 'Open', 'High', 'Low']]

# getting all the average close/last prices and storing in an array
amazon_avgs = []
for year in sorted(set(amazon_years)):
  total = 0
  # temp data set which corresponds to each year in the 5 year average
  temp = amazon.loc[amazon['Year'] == str(year)]
  close_last = list(temp['Close/Last'])
  for val in close_last:
    total += float(val[1::])

  amazon_avgs.append(total / len(close_last))

In [ ]:
# all the pre processing done for Microsoft data set
microsoft_dates = list(microsoft['Date'])
microsoft_years = []

# adding all the years stripping out the MM/DD/ part of the date
for date in microsoft_dates:
  microsoft_years.append(date[6::])

# adding the new column to data set
microsoft['Year'] = microsoft_years
microsoft = microsoft[['Year', 'Date', 'Close/Last', 'Volume', 'Open', 'High', 'Low']]

# getting all the average close/last prices and storing in an array
microsoft_avgs = []
for year in sorted(set(microsoft_years)):
  total = 0
  # temp data set which corresponds to each year in the 5 year average
  temp = microsoft.loc[microsoft['Year'] == str(year)]
  close_last = list(temp['Close/Last'])
  for val in close_last:
    total += float(val[1::])

  microsoft_avgs.append(total / len(close_last))

This code here cleans the S&P 500 price data frame to obtain the years 2018-2023 to remain consistent with the companies years chosen. This is done to get a 5 year average plot of the S&P 500 so we can see how the overall market has been doing compared to each individual company

In [ ]:
pd.options.mode.chained_assignment = None

# Cleaning/Filtering through the S&P Data to get the relevant information we need
sp_dates = list(sp_data['Date'])
sp_years = []
for date in sp_dates:
  # stripping the MM/DD part of the date to get just the years
  sp_years.append(date[-4:])

# adding the column to the dataframe to be used in later computation
sp_data['Year'] = sp_years
# converting all years to ints
sp_data['Year'] = sp_data['Year'].astype(int)
# dropping all columns that are before 2018
sp_data = sp_data[sp_data['Year'] >= 2018]

# getting the five year average price increase for the S&P 500
sp_fives = list(sp_data['Year'])
sp_avgs = []
for year in sorted(set(sp_fives)):
  total = 0
  temp = sp_data.loc[sp_data['Year'] == year]
  prices = list(temp['Price'])
  for price in prices:
    total += float(price)

  # list of averages
  sp_avgs.append(total / len(prices))

Data Pre Processing For ML and Hyothesis Testing:

In [ ]:
def process_values(money_val):
  # remove the $ sign (pre processing)
  corrected_val = str(money_val)[1:]
  return float(corrected_val)

In [ ]:
# Data Pre-Prcessing:
dataset.rename(columns = {'Close/Last':'Close'}, inplace = True)
cols = dataset.columns.tolist()
cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
dataset = dataset[cols]

dataset['Open'] = dataset['Open'].apply(process_values)
dataset['High'] = dataset['High'].apply(process_values)
dataset['Low'] = dataset['Low'].apply(process_values)
dataset['Close'] = dataset['Close'].apply(process_values)

Apple Data Preprocessing:

In [ ]:
# Split the data into a pre 2019 set and a post
dataset_train = dataset[1100:]
dataset_test = dataset[0:1100]

dataset_train = dataset_train.iloc[::-1]
dataset_test = dataset_test.iloc[::-1]

# Train Data
print(dataset_train.head())
# Test Data
print(dataset_test.head())

Amazon Data Pre Processing:

In [ ]:
# Split the data into a pre 2020  set and a post
dataset2_train = dataset2[855:]
dataset2_test = dataset2[0:855]

dataset2_train = dataset2_train.iloc[::-1]
dataset2_test = dataset2_test.iloc[::-1]

# Train Data
print(dataset2_train.head())
print()
# Test Data
print(dataset2_test.head())

These next parts involve some web scraping to collect current data to construct tables pertaining to specific attriubtes about the companies. Since we plotted the 5 year average closing/last price for each company, we now want to see if the current real time close/last price has changed. We are also showing their market cap as this also provides insights towards the companies growth

Collecting current Amazon data
* https://finance.yahoo.com/quote/AMZN/

In [ ]:
# web scraping current Amazon data off of Yahoo Finance 
amzn_url = 'https://finance.yahoo.com/quote/AMZN/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    'From': 'stopdenyingme@gmail.com'
}
ret = requests.get(amzn_url, headers=headers)

page = BeautifulSoup(ret.content, 'lxml')

# finding the table
amzn_data = pd.read_html(str(page.find_all('table')))[0]
amzn_data.columns = ['Label', 'Value']

# getting the second table
amzn_data2 = pd.read_html(str(page.find_all('table')))[1]
amzn_data2.columns = ['Label', 'Value']

# getting the previous close data and market cap data for tables
amzn_prev_close = float(amzn_data['Value'][0])
amzn_market_cap = amzn_data2['Value'][0]

Collecting current Apple data
* https://finance.yahoo.com/quote/AAPL?p=AAPL&ncid=yahooproperties_peoplealso_km0o32z3jzm

In [ ]:
# This processing is similar to the Amazon web scraping

apple_url = 'https://finance.yahoo.com/quote/AAPL?p=AAPL&ncid=yahooproperties_peoplealso_km0o32z3jzm'

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    'From': 'stopdenyingme@gmail.com'
}
ret = requests.get(apple_url, headers=headers)

page = BeautifulSoup(ret.content, 'lxml')

apple_data = pd.read_html(str(page.find_all('table')))[0]
apple_data.columns = ['Label', 'Value']

apple_data2 = pd.read_html(str(page.find_all('table')))[1]
apple_data2.columns = ['Label', 'Value']

apple_prev_close = float(apple_data['Value'][0])
apple_market_cap = apple_data2['Value'][0]

Collecting current Starbucks data
* https://finance.yahoo.com/quote/SBUX?p=SBUX&.tsrc=fin-srch



In [ ]:
starbucks_url = 'https://finance.yahoo.com/quote/SBUX?p=SBUX&.tsrc=fin-srch'

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    'From': 'stopdenyingme@gmail.com'
}
ret = requests.get(starbucks_url, headers=headers)

page = BeautifulSoup(ret.content, 'lxml')

starbucks_data = pd.read_html(str(page.find_all('table')))[0]
starbucks_data.columns = ['Label', 'Value']

starbucks_data2 = pd.read_html(str(page.find_all('table')))[1]
starbucks_data2.columns = ['Label', 'Value']

starbucks_prev_close = float(starbucks_data['Value'][0])
starbucks_market_cap = starbucks_data2['Value'][0]

Collecting current Microsoft data
* https://finance.yahoo.com/quote/MSFT?p=MSFT&.tsrc=fin-srch

In [ ]:
microsoft_url = 'https://finance.yahoo.com/quote/MSFT?p=MSFT&.tsrc=fin-srch'

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    'From': 'stopdenyingme@gmail.com'
}
ret = requests.get(microsoft_url, headers=headers)

page = BeautifulSoup(ret.content, 'lxml')

microsoft_data = pd.read_html(str(page.find_all('table')))[0]
microsoft_data.columns = ['Label', 'Value']

microsoft_data2 = pd.read_html(str(page.find_all('table')))[1]
microsoft_data2.columns = ['Label', 'Value']

microsoft_prev_close = float(microsoft_data['Value'][0])
microsoft_market_cap = microsoft_data2['Value'][0]

This is some helper code which converts all the market caps to one unit which makes it easier to sort in the table we later construct which has sorted values

In [ ]:
# This will multiply the value by a scalar based on its suffix
multipliers = {'B': 1e-3, 'T': 1}

# Helper function to convert every value to one unit (trillion) for sorting
def to_trillion(value):
  suffix = value[-1]
  mult = multipliers[suffix]
  val = float(value[:-1])
  return val * mult

This code constructs the tables we are interested in

In [ ]:
# lists of data for the data frame 
companies = ['Amazon', 'Apple', 'Starbucks', 'Microsoft']
market_caps = [amzn_market_cap, apple_market_cap, starbucks_market_cap, microsoft_market_cap]
prev_closes = [amzn_prev_close, apple_prev_close, starbucks_prev_close, microsoft_prev_close]

# creating both data frames
data = {'Company': companies, 'Market Cap': market_caps}
data2 = {'Company': companies, 'Prev Close Price': prev_closes}

# sorting values in the market cap dataframe
market_cap_df = pd.DataFrame(data)
market_cap_df['Market Cap'] = market_cap_df['Market Cap'].apply(to_trillion)
market_cap_df = market_cap_df.sort_values(by='Market Cap', ignore_index=True)
market_cap_df.columns = ['Company', 'Market Cap (Trillion)']

# sorting by previous close price
prev_close_df = pd.DataFrame(data2)
prev_close_df = prev_close_df.sort_values(by='Prev Close Price', ignore_index=True)

# Exploratory Analysis & Data Visualization

In the next few cells we are creating a plot for each company which plots their average close/last price in the last 5 years and this will be later compared with their current real time close/last price which will be in the form of a table

In [ ]:
# Creating the plot for the S&P500
plt.plot(list(sorted(set(sp_fives))), sp_avgs, color='Purple')
plt.ylabel('Average Price ($)')
plt.xlabel('Year')
plt.title('S&P500 Five Year Growth in Price')
plt.show()

In [ ]:
# Creating the plot for Apple
plt.plot(list(sorted(set(apple_years))), apple_avgs, color='blue')
plt.ylabel("Average Close/Last Price")
plt.xlabel("Year")
plt.title("Apple's Avg Close/Last Price Over the Years")
plt.show()

In [ ]:
# Creating the plot for Starbucks
plt.plot(list(sorted(set(starbucks_years))), starbucks_avgs, color='red')
plt.ylabel("Average Close/Last Price")
plt.xlabel("Year")
plt.title("Starbucks Avg Close/Last Price Over the Years")
plt.show()

In [ ]:
# Creating the plot for Amazon
plt.plot(list(sorted(set(amazon_years))), amazon_avgs, color='green')
plt.ylabel("Average Close/Last Price")
plt.xlabel("Year")
plt.title("Amazon's Avg Close/Last Price Over the Years")
plt.show()

In [ ]:
# Creating the plot for Microsoft
plt.plot(list(sorted(set(microsoft_years))), microsoft_avgs, color='orange')
plt.ylabel("Average Close/Last Price")
plt.xlabel("Year")
plt.title("Microsoft's Avg Close/Last Price Over the Years")
plt.show()

In [ ]:
# Add 'Company' column to individual company datasets
apple['Company'] = 'Apple'
starbucks['Company'] = 'Starbucks'
amazon['Company'] = 'Amazon'
microsoft['Company'] = 'Microsoft'

# Combine the data for all companies
data = pd.concat([apple, starbucks, amazon, microsoft])

# Convert 'Year' and 'Close/Last' columns to numeric
data['Year'] = pd.to_numeric(data['Year'])
data['Close/Last'] = data['Close/Last'].str.replace('$', '').astype(float)

# Set up the plot
sns.set(style='whitegrid')
plt.figure(figsize=(10, 6))

# Create the violin plot
sns.violinplot(data=data, x='Year', y='Close/Last', hue='Company', inner='quartile')
plt.xlabel('Year')
plt.ylabel('Close/Last Price')
plt.title('Close/Last Price Distribution Over the Years (All Companies)')
plt.legend(title='Company')

# Adjust the plot layout
plt.tight_layout()

# Show the plot
plt.show()

This violin plot, 

In [ ]:
plt.figure(figsize=(12, 6))

# Select every 10th date for plotting
x_ticks = apple['Date'][::100]

# Plotting Volume by Day for Apple
plt.plot(apple['Date'], apple['Volume'], label='Apple')

# Plotting Volume by Day for Starbucks
plt.plot(starbucks['Date'], starbucks['Volume'], label='Starbucks')

# Plotting Volume by Day for Amazon
plt.plot(amazon['Date'], amazon['Volume'], label='Amazon')

# Plotting Volume by Day for Microsoft
plt.plot(microsoft['Date'], microsoft['Volume'], label='Microsoft')

# plot labels and legends for visuals
plt.xlabel('Date')
plt.ylabel('Volume')
plt.title('Volume by Day - All Companies')
# Set custom x-axis ticks
plt.xticks(x_ticks, rotation=45, ha='right')  
plt.legend()
# Adjust spacing
plt.tight_layout()
plt.show()

The tables we previously constructed are being displayed here after cleaning and sorting values

In [ ]:
market_cap_df

In [ ]:
prev_close_df

In [ ]:
def plotYear(df, year, ax):
  y = df[df['Date'].str[6:] == year]['Open'].iloc[::-1].values
  x = np.arange(len(y))
  m, b = np.polyfit(x, y, 1)
  ax.plot(x, y)
  ax.plot(x, m*x+b)
  ax.set_title(year)
  ax.set_ylabel("Price")

Apple's value saw a decline in 2015 compared to previous years, due to suspicions among shareholders of iPhone market saturation.
https://time.com/4484791/apple-stock-aapl-iphone-7/

In [ ]:
f = plt.figure(figsize=(10,3))
ax = f.add_subplot(121)
ax2 = f.add_subplot(122)
plotYear(dataset, '2014', ax)
plotYear(dataset, '2015', ax2)

Amazon saw high levels of growth during 2020 due to the pandemic, which increased the value of online retail.

In [ ]:
f = plt.figure(figsize=(10,3))
ax = f.add_subplot(121)
ax2 = f.add_subplot(122)
plotYear(dataset, '2018', ax)
plotYear(dataset, '2020', ax2)

# Machine Learning and Hypothesis Testing:

In [ ]:
import seaborn as sns
import scipy
from scipy import stats
import statsmodels.api

# Hypothesis Testing:

Suppose we wish to conduct a hypothesis test in order to test the correlation between 2 different variables: trading volume and closing price. In other words, we wish to see if the price of a stock has an affect on it's volume traded. Our Null Hypothesis (H0) is the idea that stock price has no bearing on it's overall trading volume. The null hypothesis is the hypothesis that there is no signifigant correlation between 2 variables. Our Alternate Hypothesis is the idea that there is a correlation between stock price and trading value. The alternate hypothesis is a statement that contradicts the null hypothesis and suggests a significant difference or relationship between variables being studied. For our case, we will be conducting hypothesis tests to see if there is a relationship between stock price and trading volume for the companies Apple and Amazon. 


Here we will be plotting the close price of Amazon's stock (x axis) against the trading volume of the Amazon stock (y-axis). We will be fitting a regression line to this plot in order to better analyze the underlying trend of the data and visualize the overall correlation between the 2 variables. 

In [ ]:
dataset = pd.read_csv('HistoricalData_1683500505754.csv')
dataset2 = pd.read_csv('HistoricalData_1683662795655.csv')
dataset3 = pd.read_csv('HistoricalData_1683660938225.csv')

# Data Pre-Prcessing:
dataset.rename(columns = {'Close/Last':'Close'}, inplace = True)
cols = dataset.columns.tolist()
cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
dataset = dataset[cols]

dataset['Open'] = dataset['Open'].apply(process_values)
dataset['High'] = dataset['High'].apply(process_values)
dataset['Low'] = dataset['Low'].apply(process_values)
dataset['Close'] = dataset['Close'].apply(process_values)

# Data Pre-Prcessing:
dataset2.rename(columns = {'Close/Last':'Close'}, inplace = True)
cols = dataset2.columns.tolist()
cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
dataset2 = dataset2[cols]

dataset2['Open'] = dataset2['Open'].apply(process_values)
dataset2['High'] = dataset2['High'].apply(process_values)
dataset2['Low'] = dataset2['Low'].apply(process_values)
dataset2['Close'] = dataset2['Close'].apply(process_values)

# Data Pre-Prcessing:
dataset3.rename(columns = {'Close/Last':'Close'}, inplace = True)
cols = dataset3.columns.tolist()
cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
dataset3 = dataset3[cols]

dataset3['Open'] = dataset3['Open'].apply(process_values)
dataset3['High'] = dataset3['High'].apply(process_values)
dataset3['Low'] = dataset3['Low'].apply(process_values)
dataset3['Close'] = dataset3['Close'].apply(process_values)

In [ ]:
print(dataset2.head())
print('')
print(dataset3.head())

In [ ]:
# AMAZON PLOT
plot = sns.regplot(x=dataset2["Close"], y=dataset2["Volume"],
            scatter_kws = {"color": "black", "alpha": 0.5},
            line_kws = {"color": "red"},
            ci = 99)
plt.title('Close Price vs Volume For Amazon')
slope, intercept, r, plot, sterr = scipy.stats.linregress(x=plot.get_lines()[0].get_xdata(), 
                                            y=plot.get_lines()[0].get_ydata())

plt.text(100, 95, 'y = ' + str(round(intercept,3)) + ' + ' + str(round(slope,3)) + 'x')

plt.xlabel('Close Price ($)')
plt.ylabel('Volume Traded (hundreds of millions)')
plt.show()

Below we will be conducting an OLS regression. OLS regressions typically provide information on the relationship between a dependent variable and independent variables, including coefficients, standard errors, t-values, p-values, R-squared values, residuals, and diagnostic statistics. Here in this example we will be conducting an OLS Regression in order to see if there is a a p value < 0.05, which reason that the correlation between close price of a stock and volume traded is statistically signifigant. Below we see in the chart that the p-value is 0. Since 0 is < 0.05, we can reject the Null Hypothesis and accept the alternate hypothesis which we previously mentioned states tha there is a statistically signifigant correlation between close price and trading volume.

In [ ]:
ret = statsmodels.formula.api.ols(formula="Close ~ Volume", data=dataset2).fit()
print(ret.summary())

Here we will be plotting the close price of Apple's stock (x axis) against the trading volume of the Amazon stock (y-axis). We will be fitting a regression line to this plot in order to better analyze the underlying trend of the data and visualize the overall correlation between the 2 variables.  

Here we will be plotting the close price of Amazon's stock (x axis) against the trading volume of the Amazon stock (y-axis). We will be fitting a regression line to this plot in order to better analyze the underlying trend of the data and visualize the overall correlation between the 2 variables.

In [ ]:
# APPLE PLOT
plot= sns.regplot(x=dataset3["Close"], y=dataset3["Volume"],
            scatter_kws = {"color": "black", "alpha": 0.5},
            line_kws = {"color": "red"},
            ci = 99)
plt.title('Close Price vs Volume For Apple')
slope, intercept, r, plot, sterr = scipy.stats.linregress(x=plot.get_lines()[0].get_xdata(),
                                                       y=plot.get_lines()[0].get_ydata())

plt.text(90, 30000, 'y = ' + str(round(intercept,3)) + ' + ' + str(round(slope,3)) + 'x')

plt.show()

Below we will be conducting an OLS regression. OLS regressions typically provide information on the relationship between a dependent variable and independent variables, including coefficients, standard errors, t-values, p-values, R-squared values, residuals, and diagnostic statistics. Here in this example we will be conducting an OLS Regression in order to see if there is a a p value < 0.05, which reason that the correlation between close price of a stock and volume traded is statistically signifigant. Below we see in the chart that the p-value is 0. Since 0 is < 0.05, we can reject the Null Hypothesis and accept the alternate hypothesis which we previously mentioned states tha there is a statistically signifigant correlation between close price and trading volume.

In [ ]:
ret = statsmodels.formula.api.ols(formula="Close ~ Volume", data=dataset2).fit()
print(ret.summary())

# Machine Learning:

In [ ]:
training_set = dataset_train.iloc[:,1:2].values
print(training_set)
print(training_set.shape)

Fitting ARIMA model as baseline

In [ ]:
from pmdarima.arima import ARIMA

In [ ]:
model = ARIMA(order=[4, 1, 3])
model.fit(training_set)
model.summary()

In [ ]:
plt.plot(dataset.iloc[::-1].iloc[:,1:2].values, color = 'red', label = 'Actual Apple Stock Price') 
plt.plot(np.append(training_set, model.predict(n_periods = 1100)), color = 'blue', label = 'Predicted Apple Stock Price') 
plt.title('Apple Stock Price Prediction In Dollars ($)')
plt.xlabel('Time (In Days)')
plt.ylabel('Apple Stock Price')
plt.legend() 

Referenced Code and Information for the LSTM Model was take from the following sites:

https://www.analyticsvidhya.com/blog/2022/01/the-complete-lstm-tutorial-with-implementation/

https://towardsdatascience.com/simple-stock-price-prediction-with-ml-in-python-learners-guide-to-ml-76896910e2ba 

https://www.simplilearn.com/tutorials/machine-learning-tutorial/stock-price-prediction-using-machine-learning

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (0,1))
scaled_training_set = scaler.fit_transform(training_set)
scaled_training_set

In [ ]:
x_train = []
y_train = []

for i in range (60,len(dataset_train)):
  x_train.append(scaled_training_set[i-60:i,0])
  y_train.append(scaled_training_set[i,0])
x_train= np.array(x_train)
y_train= np.array(y_train)

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
regressor = Sequential()
regressor.add(LSTM(units = 50, return_sequences= True, input_shape = (x_train.shape[1], 1)))
regressor.add(Dropout (0.2))

regressor.add(LSTM(units = 50, return_sequences= True))
regressor.add(Dropout (0.2))

regressor.add(LSTM(units = 50, return_sequences=True))
regressor.add(Dropout (0.2))

regressor.add(LSTM(units = 50, return_sequences=True))
regressor.add(Dropout (0.2))

regressor.add(LSTM(units = 50))
regressor.add (Dropout (0.2))

regressor.add(Dense (units=1))

In [ ]:
regressor.compile(optimizer='adam', loss='mean_squared_error')
regressor.fit(x_train, y_train, epochs=100, batch_size=32)            

In [ ]:
actual_stock_price = dataset_test.iloc[:,1:2].values
print(dataset_test)

In [ ]:
dataset_total = pd.concat((dataset_train [ 'Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total [len(dataset_total)- len(dataset_test)-60:].values

inputs = inputs.reshape(-1,1)
inputs = scaler.transform(inputs)

x_test = []
for i in range(60,len(dataset_test)):
  x_test.append(inputs [i-60:i, 0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
predicted_stock_price = regressor.predict(x_test)
predicted_stock_price = scaler.inverse_transform(predicted_stock_price)

In [ ]:
plt.plot(actual_stock_price, color = 'red', label = 'Actual Apple Stock Price') 
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Apple Stock Price') 
plt.title('Apple Stock Price Prediction In Dollars ($)')
plt.xlabel('Time (In Days)')
plt.ylabel('Apple Stock Price')
plt.legend() 

In [ ]:
plt.plot(actual_stock_price, color = 'red', label = 'Actual Amazon Stock Price') 
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Amazon Stock Price') 
plt.title('Amazon Stock Price Prediction In Dollars ($)')
plt.xlabel('Time (In Days)')
plt.ylabel('Amazon Stock Price') 
plt.legend()